# Spectrograms

Alexandre R.J. Francois

In [ ]:
import numpy as np
from matplotlib import colormaps as mcm
import matplotlib.pyplot as plt
import librosa
import soundfile as sf

from noFFT_utils import log_frequencies, alphas_heuristic, resonate_wrapper, frequency_sweep


In [ ]:
#https://librosa.org/doc/main/recordings.html


# y, sr = sf.read(librosa.ex("brahms"))
y, sr = sf.read(librosa.ex("vibeace"))
# y, sr = sf.read(librosa.ex("sweetwaltz"))
# y, sr = sf.read(librosa.ex("libri1"))
# y, sr = sf.read(librosa.ex("libri2"))
# y, sr = sf.read(librosa.ex("libri3"))
# y, sr = sf.read(librosa.ex("robin"))
print(sr)

librosa.display.waveshow(y, sr=sr)
print(y.shape)
print(y.dtype)

# float_y = np.array(y, dtype=np.float32)
# print(float_y.dtype)

In [ ]:
fmin = 32.70
n_freqs = 100
freqs_per_octave = 12

frequencies = log_frequencies(fmin=fmin, n_freqs=n_freqs, freqs_per_octave=freqs_per_octave)
if frequencies[-1] > sr / 2:
    print("Some frequencies higher than sampling rate / 2")

alphas = alphas_heuristic(frequencies, sr=sr, k=1)
hop_length = 1
dhl = 512 # hop length for display

# print(frequencies.shape, frequencies)

In [ ]:
Rcx = resonate_wrapper(y=y, sr=sr, frequencies=frequencies, alphas=alphas, hop_length=hop_length, output_type='complex')
Rcx = Rcx.T
print(Rcx.shape, Rcx.dtype)

# compute powers
R_pows = np.abs(Rcx) ** 2
print(R_pows.shape, R_pows.dtype)

R_db = librosa.power_to_db(R_pows, ref=np.max)

# Single spectrogram
fig, ax = plt.subplots(figsize=(8, 3), dpi=300)
img = librosa.display.specshow(
    R_db[:, ::dhl],
    sr=sr,
    fmin = frequencies[0],
    hop_length=dhl,
    y_axis="cqt_hz",
    x_axis="s",
    ax=ax
)
fig.colorbar(img, ax=ax, format="%+2.f dB")


In [ ]:
# Equilizer coefficients
# Frequency sweep

eq = frequency_sweep(frequencies=frequencies, alphas=alphas, sr=sr)
# print(eq.shape, eq)

print(eq.shape, eq)

In [ ]:
# Equalized spectrograms

# Req = (eq * Rcx.T).T
# Req_pows = np.abs(Req) ** 2

Req_pows = (eq * R_pows.T).T

Req_db = librosa.power_to_db(Req_pows, ref=np.max)

# print(R_db.shape, Req_db.shape)

diff = ((1-eq) * Rcx.T).T
diff_pows = np.abs(diff) ** 2
diff_db = librosa.power_to_db(diff_pows, ref=np.max)

start = None
end = None

# spectrograms
fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(8, 8), dpi=300)
img = librosa.display.specshow(
    R_db[:, start:end:dhl],
    sr=sr,
    fmin = frequencies[0],
    hop_length=dhl,
    bins_per_octave=freqs_per_octave,
    x_axis="time",
    y_axis="cqt_hz",
    ax=ax[0],
    # vmin=-80,
    # vmax=0,
)
fig.colorbar(img, ax=[ax[0]], format="%+2.f dB")
ax[0].set(title='Raw spectrogram')
ax[0].label_outer()
img = librosa.display.specshow(
    Req_db[:, start:end:dhl],
    sr=sr,
    fmin = frequencies[0],
    hop_length=dhl,
    bins_per_octave=freqs_per_octave,
    x_axis="time",
    y_axis="cqt_hz",
    ax=ax[1],
    # vmin=-40,
    # vmax=0,
)
fig.colorbar(img, ax=[ax[1]], format="%+2.f dB")
ax[1].set(title="Equalized spectrogram")


In [ ]:
# Librosa CQT spectrogram

C = librosa.cqt(
    y=y,
    sr=sr,
    hop_length=dhl,
    fmin=fmin,
    n_bins=n_freqs,
    bins_per_octave=freqs_per_octave,
)

Cmods = np.abs(C)
C_db = librosa.amplitude_to_db(Cmods, ref=np.max)

print(C_db.shape)

detnh = 50
detns = detnh*dhl

fig, ax = plt.subplots(nrows=2, ncols=2, sharex='col', figsize=(8, 6), dpi=300)
img = librosa.display.specshow(
    C_db[:,:],
    sr=sr,
    hop_length=dhl,
    fmin=fmin,
    bins_per_octave=freqs_per_octave,
    y_axis="cqt_hz",
    x_axis="s",
    ax=ax[0][0],
)
ax[0][0].set(title="CQT")
ax[0][0].label_outer()
img = librosa.display.specshow(
    C_db[:,0:detnh],
    sr=sr,
    hop_length=dhl,
    fmin=fmin,
    bins_per_octave=freqs_per_octave,
    y_axis="cqt_hz",
    x_axis="s",
    ax=ax[0][1],
)
ax[0][1].set(title="CQT (detail)")
ax[0][1].label_outer()

librosa.display.specshow(
    R_db[:, ::dhl],
    sr=sr,
    hop_length=dhl,
    fmin=fmin,
    bins_per_octave=freqs_per_octave,
    y_axis="cqt_hz",
    x_axis="s",
    ax=ax[1][0],
)
ax[1][0].set(title="Resonate")
ax[1][0].label_outer()
librosa.display.specshow(
    R_db[:, 0:detns:dhl],
    sr=sr,
    hop_length=dhl,
    fmin=fmin,
    bins_per_octave=freqs_per_octave,
    y_axis="cqt_hz",
    x_axis="s",
    ax=ax[1][1],
)
ax[1][1].set(title="Resonate (detail)")
fig.colorbar(img, ax=ax, format="%+2.f dB")
